# Olist E-commerce Customer Segmentation Project

## Context
This project involves analyzing Olist's e-commerce data to develop customer segmentation and key performance indicators (KPIs) for a Customer Experience Dashboard. The goal is to extract actionable insights for marketing campaigns and customer behavior analysis.

## Project Objectives
1. **Implement SQL queries** to extract data for the Customer Experience Dashboard.
2. **Develop customer segmentation** to support targeted marketing campaigns.
3. **Analyze customer behavior patterns** to identify trends and opportunities.


In [1]:
# Configure Plotly to properly render in HTML exports
import plotly.io as pio
import plotly.graph_objects as go
import os

# Set the renderer for notebook display
pio.renderers.default = "notebook"

# Enable figure exports with all necessary dependencies embedded
#pio.write_html_config.include_plotlyjs = 'cdn'
#pio.write_html_config.include_mathjax = 'cdn'
#pio.write_html_config.full_html = True

# Configure global theme for consistent appearance
pio.templates.default = "plotly_white"

## 1. Data Exploration

### 1.1 Database Connection Setup
- **Objective**: Establish a connection to the SQLite database and verify the available tables.
- **Steps**:
  1. Initialize the database connection.
  2. List all available tables in the dataset.
  3. Preview the first few rows of each table to understand its structure.

In [ ]:
from src.scripts.sqlite_connector import DatabaseConnection
import os

# Get current working directory
cwd = os.getcwd()

# Initialize database connection
db_path = os.path.join(cwd, 'dataset', 'olist.db')
db = DatabaseConnection(db_path)

# Get all table names
tables = db.get_table_names()
print("Available tables:", tables)

# Read specific table
for table in tables:
    orders_df = db.read_table(table)
    display(orders_df.head())


### 1.2 Data Overview
- **Objective**: Understand the contents of the database.
- **Key Tables**:
  - **Orders**: Contains order history and delivery details.
  - **Customers**: Includes customer demographic and geographical data.
  - **Products**: Provides product details and categories.
  - **Reviews**: Contains customer satisfaction reviews.
  - **Sellers**: Includes seller information and performance metrics.

### 1.3 Query 1: Late Deliveries Analysis
- **Objective**: Identify recent orders (less than 3 months old) that were delivered with a delay of at least 3 days, excluding canceled orders.
- **Insights**:
  - Helps assess delivery performance and identify areas for improvement.

In [ ]:
query_late_deliveries = """
    WITH delivery_stats AS (
        SELECT MAX(order_purchase_timestamp) as latest_estimate
        FROM orders
        WHERE order_status != 'canceled'
    ),
    delayed_deliveries AS (
        SELECT o.*
        FROM orders o, delivery_stats d
        WHERE 
            o.order_estimated_delivery_date IS NOT NULL AND o.order_delivered_customer_date IS NOT NULL
            AND o.order_status != 'canceled'
            AND o.order_delivered_customer_date > date(o.order_estimated_delivery_date, '+3 days')
            AND o.order_purchase_timestamp BETWEEN 
                date(d.latest_estimate, '-3 months') 
                AND d.latest_estimate
    )
    SELECT * FROM delayed_deliveries;
    """
df_late_deliveries = db.execute_query(query_late_deliveries)
display(df_late_deliveries)

In [ ]:
query_last_6months_deliveries = """
    WITH delivery_stats AS (
        SELECT MAX(order_purchase_timestamp) as latest_estimate
        FROM orders
        WHERE order_status != 'canceled'
    ),
    delayed_deliveries AS (
        SELECT o.*
        FROM orders o, delivery_stats d
        WHERE 
            o.order_estimated_delivery_date IS NOT NULL AND o.order_delivered_customer_date IS NOT NULL
            AND o.order_status != 'canceled'
            AND o.order_purchase_timestamp BETWEEN 
                date(d.latest_estimate, '-6 months') 
                AND d.latest_estimate
    )
    SELECT * FROM delayed_deliveries;
    """
df_deliveries_6months = db.execute_query(query_last_6months_deliveries)
display(df_deliveries_6months)

In [ ]:
from src.scripts.visualize_late_deliveries import visualize_late_deliveries

# Create visualization for late deliveries
fig = visualize_late_deliveries(df_deliveries_6months)
fig.show()

### 1.4 Query 2: High Revenue Sellers
- **Objective**: Identify sellers who generated over 100,000 BRL in revenue from delivered orders.
- **Insights**:
  - Highlights top-performing sellers for potential partnerships or incentives.

In [ ]:
query_high_revenue = """
SELECT 
    s.seller_id,
    CAST(SUM(oi.price + oi.freight_value) AS INTEGER) as total_revenue
FROM sellers s
    JOIN order_items oi ON s.seller_id = oi.seller_id
    JOIN orders o ON oi.order_id = o.order_id WHERE o.order_status = 'delivered'
GROUP BY s.seller_id
HAVING total_revenue > 100000
ORDER BY total_revenue DESC;
"""
df_high_revenue = db.execute_query(query_high_revenue)
display(df_high_revenue)

In [ ]:
from src.scripts.visualize_top_sellers import visualize_top_sellers

# Create visualization for high revenue sellers
fig = visualize_top_sellers(df_high_revenue)
fig.show()

### 1.5 Query 3: Engaged New Sellers
- **Objective**: Identify new sellers (active for less than 3 months) who have sold more than 30 products.
- **Insights**:
  - Helps track the onboarding success of new sellers.

In [ ]:
query_new_engaged_sellers = """
WITH seller_stats AS (
    SELECT 
        s.seller_id,
        MIN(o.order_purchase_timestamp) as first_sale,
        COUNT(DISTINCT oi.order_id) as total_orders,
        COUNT(oi.product_id) as total_products
    FROM sellers s
        JOIN order_items oi ON s.seller_id = oi.seller_id
        JOIN orders o ON oi.order_id = o.order_id
    GROUP BY s.seller_id
),
latest_purchase_date AS (
    SELECT MAX(order_purchase_timestamp) as latest_purchase
    FROM orders
)

SELECT s.*
FROM seller_stats s, latest_purchase_date l
    WHERE s.total_products > 30
    AND s.first_sale >= date(l.latest_purchase, '-3 months')
ORDER BY s.total_products DESC;
"""
df_new_engaged_sellers = db.execute_query(query_new_engaged_sellers)
display(df_new_engaged_sellers)

### 1.6 Query 4: Worst Reviewed Postal Codes
- **Objective**: Identify the top 5 postal codes with the worst average review scores (minimum 30 reviews) in the last 12 months.
- **Insights**:
  - Pinpoints geographical areas with potential service or product quality issues.

In [ ]:
query_worst_postcodes = """

WITH latest_purchase_date AS (
    SELECT MAX(order_purchase_timestamp) as latest_purchase
    FROM orders
)

SELECT 
    c.customer_zip_code_prefix,
    COUNT(r.review_id) as review_count,
    ROUND(AVG(r.review_score), 2) as avg_score
FROM customers c, latest_purchase_date l
    JOIN orders o ON c.customer_id = o.customer_id
    JOIN order_reviews r ON o.order_id = r.order_id
WHERE r.review_creation_date >= date(l.latest_purchase, '-12 months')
GROUP BY c.customer_zip_code_prefix
HAVING review_count >= 30
ORDER BY avg_score ASC
LIMIT 5;
"""
df_worst_postcodes= db.execute_query(query_worst_postcodes)
display(df_worst_postcodes)

In [10]:
query_all_postcodes = """

WITH latest_purchase_date AS (
    SELECT MAX(order_purchase_timestamp) as latest_purchase
    FROM orders
)

SELECT 
    c.customer_zip_code_prefix,
    COUNT(r.review_id) as review_count,
    ROUND(AVG(r.review_score), 2) as avg_score
FROM customers c, latest_purchase_date l
    JOIN orders o ON c.customer_id = o.customer_id
    JOIN order_reviews r ON o.order_id = r.order_id
WHERE r.review_creation_date >= date(l.latest_purchase, '-12 months')
GROUP BY c.customer_zip_code_prefix
HAVING review_count >= 1
ORDER BY avg_score ASC;

"""
df_all_postcodes= db.execute_query(query_all_postcodes)

In [ ]:
from src.scripts.visualize_customer_review_bubble import create_brazil_postcode_map
# Create and display the map
fig = create_brazil_postcode_map(df_all_postcodes)
fig.show()

# 2. Data Extraction and Feature Engineering

### 2.1 RFM Features
- **Objective**: Calculate Recency, Frequency, and Monetary (RFM) metrics for each customer.
  - **Recency**: Time since the last purchase.
  - **Frequency**: Number of orders placed.
  - **Monetary**: Total spending by the customer.
- **Insights**:
  - RFM metrics are foundational for customer segmentation.


In [ ]:
query_rfm = """
WITH last_purchase_date AS (
    SELECT MAX(order_purchase_timestamp) as max_date
    FROM orders
    WHERE order_status = 'delivered'
)
SELECT 
    c.customer_id,
    MAX(o.order_purchase_timestamp) as order_purchase_timestamp,
    -- Recency
    CAST(JULIANDAY(l.max_date) - JULIANDAY(MAX(o.order_purchase_timestamp)) AS INTERGER) as recency_days,
    -- Frequency
    COUNT(o.order_id) as frequency,
    -- Monetary
    CAST(SUM(oi.price + oi.freight_value) AS INTEGER) as monetary
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
JOIN order_items oi ON o.order_id = oi.order_id
CROSS JOIN last_purchase_date l
WHERE o.order_status = 'delivered'
GROUP BY c.customer_id;
"""
df_rfm= db.execute_query(query_rfm)
display(df_rfm)

### 2.2 Customer Satisfaction Metrics
- **Objective**: Analyze customer satisfaction based on review data.
  - **Average Review Score**: Overall satisfaction level.
  - **Review Count**: Number of reviews submitted.
  - **Negative Reviews**: Count of reviews with scores ≤ 2.
- **Insights**:
  - Helps identify dissatisfied customers and areas for improvement.

In [ ]:
query_satisfaction = """
WITH customers_with_negative AS (
    SELECT DISTINCT o.customer_id, 1 as has_negative_flag
    FROM orders o
    JOIN order_reviews r ON o.order_id = r.order_id
    WHERE r.review_score <= 2
)

SELECT
    o.customer_id,
    CAST(AVG(r.review_score) AS INTEGER) as avg_review_score,
    COUNT(r.review_id) as review_count,
    COALESCE(cwn.has_negative_flag, 0) as has_negative_flag
FROM orders o
JOIN order_reviews r ON o.order_id = r.order_id
LEFT JOIN customers_with_negative cwn ON o.customer_id = cwn.customer_id
GROUP BY o.customer_id;
"""
df_satisfaction = db.execute_query(query_satisfaction)
display(df_satisfaction)

### 2.3 Customer Behavior Patterns
- **Objective**: Extract behavioral insights from customer purchase data.
  - **Product Category Diversity**: Number of unique product categories purchased.
  - **Average Delivery Time**: Time taken for orders to be delivered.
  - **Unique Sellers**: Number of sellers a customer has interacted with.
- **Insights**:
  - Provides a deeper understanding of customer preferences and habits.

In [ ]:
query_behavior = """
SELECT 
    o.customer_id,
    COUNT( p.product_category_name) as unique_categories,
    AVG(CAST(JULIANDAY(order_delivered_customer_date) - 
        JULIANDAY(order_purchase_timestamp) AS INTEGER)) as avg_delivery_time,
    COUNT( oi.seller_id) as unique_sellers
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id
JOIN products p ON oi.product_id = p.product_id
WHERE o.order_status = 'delivered'
GROUP BY o.customer_id;
"""
df_behavior= db.execute_query(query_behavior)
display(df_behavior)

## 3. Feature Analysis and Transformation

### 3.1 Merge Features
- **Objective**: Combine RFM metrics, satisfaction metrics, and behavior patterns into a unified dataset for analysis.
- **Outcome**:
  - A consolidated DataFrame ready for further analysis and modeling.

In [ ]:
from src.classes.feature_analysis import FeatureAnalysis
from src.classes.feature_engineering import FeatureEngineering

# Initialize with existing query results
fe = FeatureEngineering(df_rfm, df_satisfaction, df_behavior)
features_df = fe.combine_features()



# Plot results
fa = FeatureAnalysis(features_df, columns_to_exclude=['order_id', 'customer_id'])
dist_plot = fa.plot_distributions()
dist_plot.show()


### 3.2 Feature Distribution Analysis
- **Objective**: Explore the distribution of features to identify patterns, outliers, and preprocessing needs.
- **Steps**:
  1. **Statistical Summaries**: Compute mean, median, standard deviation, etc.
  2. **Distribution Plots**: Visualize feature distributions using histograms or density plots.
  3. **Outlier Detection**: Use box plots to identify extreme values.
- **Outcome**:
  - Informs decisions on scaling, normalization, and handling outliers.

In [ ]:
from src.scripts.analyze_numerical_outliers import analyze_outliers_with_multiple_methods

# Analyze all numeric variables with different outlier detection methods
all_summaries, all_cleaned_dfs = analyze_outliers_with_multiple_methods(features_df)

### 3.3 Box Plot Analysis
- **Objective**: Visualize feature distributions and detect outliers.
- **Steps**:
  1. Plot box plots for numerical features.
  2. Highlight features with significant outliers.
- **Outcome**:
  - Guides feature scaling and normalization.

In [ ]:
from src.scripts.visualize_numerical_outliers import create_interactive_outlier_visualization

# Create the interactive outlier visualization
summary_df, df_cleaned = create_interactive_outlier_visualization(all_cleaned_dfs['Z-score (±3)'])

### 3.4 RFM dashboard review after normalization
- **Objective**: Visualize RFM feature distributions after removal of outliers.


In [ ]:
# Create a hybrid dataframe with cleaned continuous variables but original flags/counts
hybrid_df = all_cleaned_dfs['Z-score (±3)'].copy()

# List of columns to restore from original data
categorical_cols = ['has_negative_flag'] 
count_cols = ['review_count', 'unique_categories', 'frequency', 'unique_sellers']

# Replace the values with originals
for col in categorical_cols + count_cols:
    if col in features_df.columns and col in hybrid_df.columns:
        hybrid_df[col] = features_df[col]

# Use the hybrid dataframe for visualization
fa = FeatureAnalysis(hybrid_df, columns_to_exclude=['customer_id'])
dist_plot = fa.plot_distributions()
dist_plot.show()


### 3.3 Feature Correlations
- **Objective**: Analyze relationships between features using a correlation matrix.
- **Steps**:
  1. Compute the correlation matrix for numerical features.
  2. Visualize the lower triangle of the matrix using a heatmap.
- **Outcome**:
  - Identifies highly correlated features and potential redundancies.

In [ ]:
# Plot results
corr_plot = fa.plot_correlation_matrix()
corr_plot.show()

## 4. Implement clustering algorithms

# 4.1 Feature Transformations for Clustering

The feature transformation process implements a sophisticated approach to prepare customer data for clustering, ensuring optimal algorithm performance while maintaining interpretability:

## Transformation Components

1. **Data Preparation**
   - Set customer_id as the index to maintain customer identity throughout the analysis
   - Removed temporal variables (order_purchase_timestamp) that aren't relevant for segmentation
   - Created a clean foundation for applying consistent transformations

2. **Specialized Transformation Strategy**
   - Implemented the `GenericFeatureTransformer` to handle different variable types appropriately
   - Applied specific transformation methods to three distinct variable categories:
     * **Numerical variables**: Applied robust scaling to minimize outlier impact
     * **Categorical indicators**: Properly encoded the has_negative_flag variable
     * **Count variables**: Applied special handling to preserve the count nature while standardizing scale

3. **Count Variable Preservation**
   - Recognized that count variables (review_count, unique_categories, frequency, unique_sellers) have special properties
   - Implemented specialized transformations that maintain their inherent characteristics
   - Ensured that zero counts remain meaningful in the transformed space

4. **Reversible Transformations**
   - Stored transformation parameters to enable inverse transformation
   - Created capability to convert clusters back to original feature space
   - Ensured business stakeholders can understand results in familiar metrics

## Outcome

The transformation process creates a dataset that:

- Places all variables on comparable scales for distance-based clustering algorithms
- Preserves the inherent structure and relationships between variables
- Minimizes the impact of outliers without losing their information
- Maintains the ability to interpret results in the original business context

This approach balances mathematical optimization with business interpretability, enabling both effective clustering and actionable insights from the resulting segments.

In [20]:
from src.classes.feature_transformation import GenericFeatureTransformer

#Set customer_id as index before transformation
hybrid_df = hybrid_df.set_index('customer_id')
hybrid_df = hybrid_df.drop(columns=['order_purchase_timestamp'], errors='ignore')

# Create and apply the generic transformer
transformer = GenericFeatureTransformer()
transformed_df = transformer.fit_transform(
    df=hybrid_df,
    categorical_cols=['has_negative_flag'],
    count_cols=['review_count', 'unique_categories', 'frequency', 'unique_sellers']
)

# Get original data back when needed
original_df = transformer.inverse_transform(transformed_df)


# 4.2 Clustering Analysis with Advanced Feature Importance

The customer segmentation implementation uses a sophisticated dual-model approach combining PCA transformation with robust feature importance analysis:

## Analysis Components

1. **PCA Component Analysis**
   - Applied dimensionality reduction to visualize high-dimensional customer data
   - Created explained variance plots to identify optimal components (87% variance captured in top 3 components)
   - Generated biplots showing feature loadings on principal components for interpretability
   - Identified which features contribute most to each principal component

2. **Optimal Cluster Determination**
   - Implemented optimized elbow method with silhouette scoring
   - Analyzed inertia and silhouette metrics to identify the ideal number of clusters
   - Selected 3 clusters based on the balance between complexity and interpretability

3. **Dual-Model K-Means Implementation**
   - Trained PCA-based model for visualization (using 3D PCA components)
   - Trained feature-space model for interpretability (using original features)
   - Maintained both models to maximize insights while enabling visualization

4. **Feature Importance Analysis**
   - Applied permutation importance to identify cluster-defining features
   - Used silhouette score reduction as the importance metric
   - Visualized with error bars to show consistency of importance scores
   - Sampling techniques applied for computational efficiency

## Outcome

The analysis reveals distinct customer segments with clear behavioral differences, enabling targeted marketing strategies based on:

- Recency, frequency, and monetary (RFM) profile of each segment
- Satisfaction levels and complaint patterns
- Product category diversity and seller engagement patterns

The implementation balances computation speed and insight depth, providing actionable customer segmentation with clear understanding of which features define each segment.

In [ ]:
import importlib
import src.classes.cluster_analysis
importlib.reload(src.classes.cluster_analysis)
from src.classes.cluster_analysis import ClusteringAnalysis

# Initialize clustering with the transformer for inverse transform capability
ca = ClusteringAnalysis(
    df=transformed_df, 
    transformer=transformer
)

In [ ]:
# Generate elbow plot to determine optimal number of clusters
elbow_plot = ca.plot_elbow(range(2, 11))
elbow_plot.show()

In [ ]:
# Visualize PCA explained variance to select optimal components
fig_pca_variance = ca.plot_pca_explained_variance(max_components=15)
fig_pca_variance.show()

In [ ]:
# Create PCA biplot to visualize feature loadings on principal components
fig_biplot = ca.plot_pca_biplot(n_features=5)
fig_biplot.show()

In [ ]:
# Visualize feature importance in principal components
fig_pca_importance = ca.plot_pca_feature_importance(n_components=4)
fig_pca_importance.show()

In [ ]:
# Run clustering with a different number of clusters (e.g., 5)
ca.fit_kmeans(n_clusters=4)

# Create and show silhouette visualization to evaluate cluster quality
silhouette_fig = ca.plot_silhouette_plotly()
silhouette_fig.show()

In [ ]:
# Create and display circle-based intercluster visualization
intercluster_fig = ca.plot_intercluster_distance_circles()
intercluster_fig.show()

In [ ]:
# Create and display circle-based intercluster visualization
kmeansfeature_fig = ca.plot_kmeans_feature_importance()
kmeansfeature_fig.show()

## Understanding the Clustering Approach

The clustering implementation uses a sophisticated dual-model approach:

1. **PCA-based Clustering Model**:
   - Used for the elbow method and visualizations
   - Applied to dimensionally-reduced data (PCA components)
   - Advantages: Better visualization, handles correlated features

2. **Original Feature Space Clustering Model**:
   - Used for feature importance analysis
   - Applied directly to transformed features (not PCA)
   - Advantages: Direct interpretability of feature effects

This dual approach explains why:
- The elbow plot may suggest different optimal clusters than silhouette analysis
- Feature importance relates to the original features, not PCA components
- We can both visualize clusters effectively and understand feature contributions## Understanding the Clustering Approach

The clustering implementation uses a sophisticated dual-model approach:

1. **PCA-based Clustering Model**:
   - Used for the elbow method and visualizations
   - Applied to dimensionally-reduced data (PCA components)
   - Advantages: Better visualization, handles correlated features

2. **Original Feature Space Clustering Model**:
   - Used for feature importance analysis
   - Applied directly to transformed features (not PCA)
   - Advantages: Direct interpretability of feature effects

This dual approach explains why:
- The elbow plot may suggest different optimal clusters than silhouette analysis
- Feature importance relates to the original features, not PCA components
- We can both visualize clusters effectively and understand feature contributions

In [ ]:
# Define your custom segment configuration - adjust thresholds based on business requirements
custom_segment_config = {
    # Recency thresholds (in days)
    'recency': {
        'active': 45,      # 0-45 days: "Active" (adjusted from default 30)
        'recent': 120,     # 46-120 days: "Recent" (adjusted from default 90)
        # > 120 days: "Inactive"
    },
    
    # Frequency thresholds (number of orders)
    'frequency': {
        'frequent': 2.5,   # > 2.5 orders: "Frequent" (adjusted from default 3)
        'returning': 1.2,  # > 1.2 orders: "Returning" (adjusted from default 1.5)
        # <= 1.2 orders: "One-time"
    },
    
    # Monetary thresholds
    'monetary': {
        'high_value': 150,  # > $150: "High-value" (fixed value instead of default 'mean')
        # <= $150: "Standard-value" 
    },
    
    # Satisfaction thresholds (review scores)
    'satisfaction': {
        'very_satisfied': 4.7,  # >= 4.7: "Very Satisfied" (adjusted from default 4.5)
        'satisfied': 4.2,       # >= 4.2: "Satisfied" (adjusted from default 4.0)
        'neutral': 3.5,         # >= 3.5: "Neutral" (adjusted from default 3.0)
        # < 3.5: "Unsatisfied"
    }
}


from src.scripts.cluster_dashboard import create_sales_dashboard
import numpy as np

# Store the original dataframe in the transformer for reference
transformer.original_df = hybrid_df

# Get the cluster labels from the clustering model
labels = ca.kmeans_results['labels']
print(f"Found {len(labels)} labels across {len(np.unique(labels))} clusters")

# Create the dashboard with custom segment configuration
dashboard = create_sales_dashboard(labels, transformed_df, transformer, custom_segment_config)

# Display the components
for name, fig in dashboard.items():
    fig.show()

## 5. Evaluate Segment Stability
- **Objective**: Assess the stability of customer segments over time.
- **Steps**:
  1. Validate feature consistency across time periods.
  2. Compute stability scores for each segment.
  3. Visualize stability trends.
- **Outcome**:
  - Ensures segments remain meaningful and actionable over time.


In [ ]:
from src.classes.stability_analysis import ClusterStabilityAnalysis
import pandas as pd

# Prepare the original dataframe with dates for temporal analysis
# We need to merge the order_purchase_timestamp back in
orders_df = db.read_table('orders')
orders_df['order_purchase_timestamp'] = pd.to_datetime(orders_df['order_purchase_timestamp'])

# Extract just the customer_id and timestamp columns
orders_date_df = orders_df[['customer_id', 'order_purchase_timestamp']]

# Group by customer_id and get the most recent order date
customer_dates = orders_date_df.groupby('customer_id').agg(
    order_purchase_timestamp=('order_purchase_timestamp', 'max')
).reset_index()

# Merge with the original dataframe (use the index which should be customer_id)
original_df_with_dates = hybrid_df.copy()
original_df_with_dates = original_df_with_dates.reset_index()
original_df_with_dates = pd.merge(
    original_df_with_dates,
    customer_dates,
    on='customer_id',
    how='left'
)

# Create a mapping between positions and original customer IDs
original_customer_ids = original_df_with_dates['customer_id'].values
index_mapping = {i: customer_id for i, customer_id in enumerate(original_customer_ids)}

# Reindex transformed_df with original customer IDs
transformed_df_with_ids = transformed_df.copy()
transformed_df_with_ids.index = pd.Index([index_mapping.get(i, i) for i in transformed_df.index])


# Initialize the stability analysis
stability_analyzer = ClusterStabilityAnalysis(
    df=transformed_df_with_ids,
    transformer=transformer,
    original_df_with_dates=original_df_with_dates
)

# Evaluate bootstrap stability (random sampling approach)
bootstrap_results = stability_analyzer.evaluate_bootstrap_stability(
    n_clusters=4,  # Same number used in your clustering
    n_iterations=20,  # Number of bootstrap samples
    sample_fraction=0.8  # Sample 80% of data each time
)

# Display bootstrap stability results
bootstrap_results['figure'].show()


# Evaluate cross-period stability (works even without customer overlap)
cross_period_results = stability_analyzer.evaluate_cross_period_stability(
    n_clusters=4,
    period='quarter',
    eval_sample_size=1000,
    min_customers_per_period=10  # Lower threshold for testing
)

# Display results
cross_period_results['figure'].show()
cross_period_results['figure_counts'].show()


## 6. Maintenance Recommendations